# 🧐 The Organizing Table

<img src=images/organizing-table-architecture.png>

# 🤝 Seperating Responsibility

## ⚡ Delegates & Events in C#

### Delegate Revision

In [ ]:
delegate int MathOperation(int x, int y);

int Add1(int x, int y) => x + y;
var operation1 = new MathOperation(Add1);
operation1(3, 4) // operation.Invoke(3, 4); operation?.Invoke

In [ ]:
delegate int MathOperation(int x, int y);
MathOperation operation2 = delegate (int x, int y) { return x - y; }; // anonymous / inline
operation2(4, 3)

In [ ]:
Action<string> log = s => Console.WriteLine(s);
Func<int, int, int> subtract = (x, y) => x - y;
log("Subtracting 3 from 4");
subtract(4, 3)

In [ ]:
var square = (int x) => x * x; // infererence
square(4)

- Builders, Factories
- Callbacks, Lazy loadings
- Control and Inversion of Control
- Multicasting
- Async Programming before async/await

In [ ]:
void FirstHandler(string message) => Console.WriteLine($"[1] {message}");
void SecondHandler(string message) => Console.WriteLine($"[2] {message}");

delegate void Notify(string message);

Notify notify = FirstHandler;
notify += SecondHandler;

notify("Hello, delegates!");

notify -= SecondHandler;
notify("After removing SecondHandler.");

In [ ]:
//Async; but will not work here in Notebook setting
using System.Threading;

delegate void LongProcess();

void Process()
{
    Console.WriteLine($"Running Process in {Thread.CurrentThread.ManagedThreadId}");
    Thread.Sleep(5000); //simulating
}

LongProcess lp = Process;
var handler = lp.BeginInvoke(null, null);
Console.WriteLine($"Running Main in {Thread.CurrentThread.ManagedThreadId}");
Thread.Sleep(3000); //simulating bit faster
lp.EndInvoke(handler);

### Events

<img src=images/event-emitter-listener.png width=800>

In [ ]:
delegate void StringDelegate(string s);

class Emitter
{
    public event StringDelegate OnStringEvent;

    public void SomeAction(string p)
    {
        // may be do something
        if (null != OnStringEvent) OnStringEvent($"Called with {p}");
        // may be do something more
    }
}

void print(string s) => Console.WriteLine($"Print Method: {s}");

var e = new Emitter();
e.SomeAction("1");

// subscribe
e.OnStringEvent += print;
e.OnStringEvent += s => Console.WriteLine($"Delegate/Inline Listener: {s}");
e.SomeAction("2");

// unsubscribe
e.OnStringEvent -= print;
e.SomeAction("3");

__Multicasting and Events__

Events in C# are build on top of delegates. When you declare an event, it uses a delegate as its underlying mechanism, and thus supports multicasting
- Events restrict direct access to the delegate's invocation list. You cannot invoke the delegate directly; only the class defining the event can
- This restriction ensures encapsulation and avoids unintended external modifications.

__Key Difference__

- Delegates: Allow full control over adding, removing, and invoking methods.
- Events: Limit control to adding and removing handlers; invocation is restricted to the event's defining class. 👈

In [ ]:
using System.IO;

class StringEventArgs : EventArgs
{
    public string StringValue { get; set; } = null;
}

class FileProcessor
{
    //public event EventHandler OnLineRead; // 1.0
    public event EventHandler<StringEventArgs> OnLineRead; // 1.1

    string filePath = null;
    public FileProcessor(string filePath)
    {
        if (!File.Exists(filePath)) throw new ArgumentException();
        this.filePath = filePath;
    }

    public void ReadLineByLine()
    {
        using var sr = new StreamReader(this.filePath);

        string line;
        while ((line = sr.ReadLine()) != null)
        {
            if (line.Length > 0 && null != this.OnLineRead) // we are checking if someone is interested
                this.OnLineRead(this, new StringEventArgs { StringValue = line });
        }
    }
}

string text = "";

void accumulator(object sender, StringEventArgs args)
{
    // in 1.0 without generics we had to cast
    text += args.StringValue;

    // if we crash here ? what will happen to sender ? 👈
    // if there are multiple listeners ?
}

var f = new FileProcessor("input.txt");
f.OnLineRead += accumulator;        // operator overloading
f.OnLineRead += (s, e) => Console.WriteLine(e.StringValue);
f.ReadLineByLine();

Console.WriteLine(text);

text = "all gone";
f.OnLineRead -= accumulator;
f.ReadLineByLine();
Console.WriteLine(text);

### Delegates vs Events

- Delegates are more general-purpose and can be used in various scenarios
- Events are specifically designed for notifications and provide a more structured and controlled way for objects to communicate with each other

# 🌐 Event Driven Thinking

- Events as state changes (e.g., OrderPlaced, PaymentProcessed)
- Difference between commands (do this) and events (this happened)
- Event characteristics
    - Immutable (facts of the past)
    - Self-contained (all data needed for processing)
    - Named in past tense (UserRegistered, not RegisterUser)

A remote procedure call (RPC) is a protocol that allows a computer program to run a function on another computer or server. RPCs are used in distributed computing and are often used to call remote functions on a server

<img src=images/rpc.png>

In [ ]:
record RequestProperties(string CorrelationId, string ReplyTo);

string correlationId = Guid.NewGuid().ToString();
var props = new RequestProperties(correlationId, "reply-to-me-here");

// Fire and forget
// Send Message

// Fire and forget; but you have a seperate "listener"
// Send Message
// Optionally let listener know we are expecting a reply....or it is always ready

// RPC
// Send Message
// Wait till you are replied or time out
// Process Reply

<img src=images/message-queueing-distributed.png>